In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [22]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [23]:
system_message = "You are a helpful assistant that responds in markdown"

In [8]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
import gradio as gr

# Função de autenticação
def autenticar(usuario, senha):
    if usuario == "" and senha == "":
        return gr.update(visible=False), gr.update(visible=True), "✅ Login realizado com sucesso!"
    else:
        return gr.update(visible=True), gr.update(visible=False), "❌ Usuário ou senha inválidos."

# Função do chat
def chat_fn(message, history):
    messages = [{"role": "system", "content": system_message}]

    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})

    messages.append({"role": "user", "content": message})

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

with gr.Blocks() as demo:
    # Área de login
    with gr.Column(visible=True) as login_area:
        gr.Markdown("## 🔒 Login")
        usuario_input = gr.Textbox(label="Usuário")
        senha_input = gr.Textbox(label="Senha", type="password")
        login_button = gr.Button("Entrar")
        mensagem_login = gr.Textbox(label="", interactive=False)

    # Área do chat
    with gr.Column(visible=False) as chat_area:
        chat_interface = gr.ChatInterface(fn=chat_fn)

    # Ligar botão de login
    login_button.click(
        autenticar,
        inputs=[usuario_input, senha_input],
        outputs=[login_area, chat_area, mensagem_login]
    )

demo.launch(share=True)
